In [1]:
# # Install with pip install firecrawl-py
from firecrawl import FirecrawlApp
import requests
import json
import os 
from tiktoken import encoding_for_model
from supabase import create_client, Client
from dotenv import load_dotenv  

load_dotenv()

FIRECRAWL_API = os.getenv("FIRECRAWL_API_KEY")
app = FirecrawlApp(api_key=FIRECRAWL_API)

async def count_tokens(text, model="gpt-4o"):
    encoder = encoding_for_model(model)
    tokens = encoder.encode(text)
    return len(tokens)

JINA_API_KEY = os.getenv('JINA_API_KEY')
async def get_embedding(text):
    """ JINA EMBEDDINGS """
    url = 'https://api.jina.ai/v1/embeddings'
    headers = {
        'Content-Type': 'application/json',
        'Authorization': f'Bearer {JINA_API_KEY}'
    }

    data = {
        "model": "jina-embeddings-v3",
        "task": "retrieval.passage",
        "dimensions": 1024,
        "late_chunking": False,
        "embedding_type": "float",
        "input": text
    }
    response = requests.post(url, headers=headers, data=json.dumps(data))
    return response.json()#['data'][0]['embedding']

async def sliding_window_chunking(text, max_window_size=900, overlap=200):
    encoder = encoding_for_model("gpt-4o")  # Use the same model as in count_tokens
    tokens = encoder.encode(text)
    chunks = []
    start = 0
    while start < len(tokens):
        end = start + max_window_size
        chunk_tokens = tokens[start:end]
        chunk = encoder.decode(chunk_tokens)
        chunks.append(chunk)
        start += max_window_size - overlap
    return chunks

SUPABASE_URL = os.getenv("SUPABASE_URL")
SUPABASE_SERVICE_ROLE_KEY = os.getenv("SUPABASE_SERVICE_ROLE_KEY")
SUPABASE_ANON_KEY = os.getenv("SUPABASE_KEY")

supabase: Client = create_client(SUPABASE_URL, SUPABASE_SERVICE_ROLE_KEY)

# map_result = app.map_url('periperi.co.uk/', params={
# 	'includeSubdomains': True
# })

async def insert_to_db(data):
    print("inserting to db")
    supabase.table('user_web_data').insert(data).execute()



In [2]:
peri_sites = ["https://www.periperi.co.uk",
 "https://www.periperi.co.uk/gallery",
 "https://www.periperi.co.uk/party-organisers",
 "https://www.periperi.co.uk/marquee-management",
 "https://www.periperi.co.uk/event-production",
 "https://www.periperi.co.uk/venue-hire",
 "https://www.periperi.co.uk/wedding-planners",
 "https://www.periperi.co.uk/events-planners",
 "https://www.periperi.co.uk/event-caterers",
 "https://www.periperi.co.uk/party-planner",
 "https://www.periperi.co.uk/marquee-hire",
 "https://www.periperi.co.uk/corporate-events",
 "https://www.periperi.co.uk/private-festival-organiser",
 "https://www.periperi.co.uk/event-caterers/cocktails",
 "https://www.periperi.co.uk/event-planning-brochure",
 "https://www.periperi.co.uk/sitemap",
 "https://www.periperi.co.uk/event-production/event-logistics",
 "https://www.periperi.co.uk/marquee-hire/corporate-marquee",
 "https://www.periperi.co.uk/venue-hire/london-venues",
 "https://www.periperi.co.uk/marquee-hire/marquee-wedding",
 "https://www.periperi.co.uk/event-production/event-theming",
 "https://www.periperi.co.uk/event-caterers/party-caterers",
 "https://www.periperi.co.uk/corporate-events/conference-organisers",
 "https://www.periperi.co.uk/event-production/event-av",
 "https://www.periperi.co.uk/venue-hire/venue-search",
 "https://www.periperi.co.uk/party-planner/party-organiser",
 "https://www.periperi.co.uk/event-production/event-styling",
 "https://www.periperi.co.uk/venue-hire/marquee-venues",
 "https://www.periperi.co.uk/wedding-planners/wedding-catering",
 "https://www.periperi.co.uk/event-caterers/creative-catering",
 "https://www.periperi.co.uk/party-planner/party-planners",
 "https://www.periperi.co.uk/venue-hire/unique-venues",
 "https://www.periperi.co.uk/wedding-planners/wedding-planning",
 "https://www.periperi.co.uk/events-planners/event-organisers",
 "https://www.periperi.co.uk/corporate-events/awards-organiser",
 "https://www.periperi.co.uk/event-caterers/bespoke-catering",
 "https://www.periperi.co.uk/venue-hire/venue-finder",
 "https://www.periperi.co.uk/party-planner/party-venues",
 "https://www.periperi.co.uk/event-production/festival-organiser",
 "https://www.periperi.co.uk/wedding-planners/wedding-planner",
 "https://www.periperi.co.uk/event-caterers/event-catering",
 "https://www.periperi.co.uk/wedding-planners/wedding-organiser",
 "https://www.periperi.co.uk/marquee-hire/hire-marquee",
 "https://www.periperi.co.uk/events-planners/event-planner",
 "https://www.periperi.co.uk/party-planner/themed-events",
 "https://www.periperi.co.uk/event-production/production-company",
 "https://www.periperi.co.uk/party-planner/party-marquee",
 "https://www.periperi.co.uk/events-planners/events-planner-london","https://www.periperi.co.uk/gallery/party-planner-uk-gallery","https://www.periperi.co.uk/wedding-planners/wedding-marquee-hire","https://www.periperi.co.uk/party-planner/party-planner-london","https://www.periperi.co.uk/events-planners/corporate-event-planner","https://www.periperi.co.uk/party-planner/birthday-party-planner","https://www.periperi.co.uk/marquee-hire/luxury-marquee-hire","https://www.periperi.co.uk/corporate-events/christmas-party-planner","https://www.periperi.co.uk/events-planners/party-planner-uk","https://www.periperi.co.uk/corporate-events/corporate-marquee-hire","https://www.periperi.co.uk/wedding-planners/wedding-planner-london","https://www.periperi.co.uk/events-planners/party-organiser-uk","https://www.periperi.co.uk/marquee-hire/marquee-for-hire","https://www.periperi.co.uk/corporate-events/corporate-event-companies","https://www.periperi.co.uk/event-production/event-planning-company","https://www.periperi.co.uk/marquee-hire/party-marquee-hire","https://www.periperi.co.uk/event-caterers/high-end-caterers","https://www.periperi.co.uk/corporate-events/charity-event-organiser","https://www.periperi.co.uk/marquee-hire/marquee-hire-london","https://www.periperi.co.uk/event-caterers/event-caterers-london","https://www.periperi.co.uk/venue-hire/international-venue-finder","https://www.periperi.co.uk/event-production/event-organisers-uk","https://www.periperi.co.uk/party-planner/luxury-party-planner","https://www.periperi.co.uk/event-caterers/bespoke-catering-london","https://www.periperi.co.uk/event-inspiration","https://www.periperi.co.uk/our-services","https://www.periperi.co.uk/gallery/marquees","https://www.periperi.co.uk/gallery/catering","https://www.periperi.co.uk/our-insurance","https://www.periperi.co.uk/gallery/weddings","https://www.periperi.co.uk/narnia-themed-corporate-event-case-study","https://www.periperi.co.uk/sx-launch","https://www.periperi.co.uk/gallery/event-production","https://www.periperi.co.uk/corporate-events/testimonials","https://www.periperi.co.uk/health-safety-analysis","https://www.periperi.co.uk/join-our-team","https://www.periperi.co.uk/gallery/corporate-events","https://www.periperi.co.uk/terms-and-conditions","https://www.periperi.co.uk/external-contractor-management","https://www.periperi.co.uk/rubec-house-2024","https://www.periperi.co.uk/event-inspiration/page/2","https://www.periperi.co.uk/clearscore-corporate-event-showcase","https://www.periperi.co.uk/bar-alcohol-licensing-management","https://www.periperi.co.uk/country-wedding-case-study","https://www.periperi.co.uk/event-inspiration/page/3","https://www.periperi.co.uk/greatest-showman-circus-theme","https://www.periperi.co.uk/venue-hire/corporate-venues","https://www.periperi.co.uk/our-proposals-planning-documents","https://www.periperi.co.uk/our-price-fees-guide","https://www.periperi.co.uk/power-electrical-equipment-management","https://www.periperi.co.uk/catering-catering-equipment-management","https://www.periperi.co.uk/greatest-show-themed-event","https://www.periperi.co.uk/oktoberfest-themed-marquee-case-study","https://www.periperi.co.uk/wedding-planners/wedding-planners-london","https://www.periperi.co.uk/event-inspiration/category/party-inspiration","https://www.periperi.co.uk/claridges-charity-dinner-case-study","https://www.periperi.co.uk/event-inspiration/category/wedding-inspiration","https://www.periperi.co.uk/support-from-our-event-team","https://www.periperi.co.uk/bespoke-nightclub-marquee-on-a-slope","https://www.periperi.co.uk/pdf/periperi-event-manager.pdf","https://www.periperi.co.uk/themed-corporate-marquee-with-paddy-mcguinness","https://www.periperi.co.uk/event-management-equipment-safety-kit-hire","https://www.periperi.co.uk/pdf/periperi-trainee-event-manager.pdf","https://www.periperi.co.uk/angela-hartnett-private-banquet-menu-design-case-study","https://www.periperi.co.uk/in-depth-guide-to-what-is-included-in-our-event-management-fee","https://www.periperi.co.uk/wp-content/uploads/2019/02/Peri-Peri-Public-Liability-Prof-Indemnity-2018.pdf","https://www.periperi.co.uk/wp-content/uploads/2019/02/Peri-Peri-Public-Liability-Cert-2018.pdf"]

In [3]:
# sb_insert = {
#     "url": "",
#     "header": "",
#     "content": "",
#     "token_count": 0,
#     "jina_embedding": "",
#     "user_id": "user_2mmXezcGmjZCf88gT2v2waCBsXv"
# }

# go = False
# for site in peri_sites:
#     if site == "https://www.periperi.co.uk/clearscore-corporate-event-showcase":
#         go = True
#     print(site)
    
#     if go:
#         try:
#             response = app.scrape_url(url=site, params={
#                 'formats': [ 'markdown' ],
#                 'waitFor': 1000
#             })

#             content = [item for item in response['markdown'].split('\n\n') if not item.startswith('[![]')]
#             content = "\n\n".join(content)
#             header = "## Title: " + response['metadata']['title'] + " ## Description: " + response['metadata']['description']
#             chunks = await sliding_window_chunking(content)

#             for chunk in chunks: 
#                 print(f"processing chunk {chunks.index(chunk)} of {len(chunks)}")
#                 sb_insert['url'] = site
#                 sb_insert['header'] = header
#                 sb_insert['content'] = chunk
#                 chunk = header + chunk
#                 jina_response = await get_embedding(chunk)
#                 sb_insert['jina_embedding'] = jina_response['data'][0]['embedding']
#                 sb_insert['token_count'] = jina_response['usage']['total_tokens']

#                 await insert_to_db(sb_insert)
#         except KeyError as e:
#             print(f"KeyError occurred for site {site}: {str(e)}")
#             continue


In [6]:
async def get_user_web_data(user_id):
    response = supabase.table('user_web_data') \
        .select('*') \
        .eq('user_id', user_id) \
        .execute()
    
    return response.data

# Usage
user_id = "user_2mmXezcGmjZCf88gT2v2waCBsXv"
user_web_data = await get_user_web_data(user_id)

headers = [] 
for head in user_web_data:
    headers.append(head['header'])

# Create a set from the headers list to remove duplicates
unique_headers = set(headers)

# Convert the set back to a list
unique_headers_list = list(unique_headers)

Number of records: 253


In [9]:
unique_headers_list
# Join the list elements into a single string
unique_headers_string = " ".join(unique_headers_list)
# You can now use this string with the count_tokens function
token_count = await count_tokens(unique_headers_string)
print(f"Total token count: {token_count}")



Total token count: 3259


In [ ]:
from services.chat.chat import llm_response

sys_prompt_sys_gen = """
You are an AI system designed to analyze business data and create effective system prompts for conversational AI agents. Your task is to craft a system prompt for an AI that will engage with prospects, understand their needs, and guide them through the prospecting and qualifying stages before handing off to a human colleague. Follow these guidelines:

Your output must ONLY contain the system prompt, with no prefix, nor introductions. 

1. Analyze the provided business data thoroughly, focusing on:
   - Company mission, vision, and core values
   - Target market and ideal customer profiles
   - Key products or services and their unique selling points
   - Standard qualifying criteria for prospects
   - Company tone and communication style

2. Create a system prompt that includes:
   - A clear definition of the AI agent's role and purpose
   - Essential company information and values
   - Guidelines for engaging with prospects professionally and empathetically
   - Instructions for identifying and qualifying leads based on company criteria
   - Directions for gathering key information from prospects
   - Protocols for when and how to hand off to human colleagues
   - Ethical guidelines and privacy considerations

3. Ensure the prompt emphasizes:
   - Building rapport and trust with prospects
   - Active listening and asking probing questions
   - Adapting communication style to different prospect personas
   - Providing value and information tailored to prospect needs
   - Respecting boundaries and not being overly pushy

4. Include instructions for the AI to:
   - Introduce itself as an AI assistant
   - Be transparent about its capabilities and limitations
   - Know when to escalate complex inquiries to human team members

5. Incorporate language that promotes:
   - Consistency with the company's brand voice
   - Flexibility in conversation flow
   - Respect for prospect's time and preferences

6. Add safeguards to prevent the AI from:
   - Making promises or commitments beyond its authority
   - Sharing sensitive or confidential information
   - Engaging in unethical or manipulative sales tactics

Your output must ONLY contain the system prompt, with no prefix, nor introductions. 

Craft the system prompt to be concise yet comprehensive, focusing on essential information that will guide the conversational AI in effectively engaging prospects and qualifying leads.

"""

sys_prompt_periperi = await llm_response(sys_prompt_sys_gen, unique_headers_string)

In [17]:
""" the system prompt gives tasks without clear instructions """
sys_prompt_periperi.split("\n\n")

['You are an AI assistant for Peri Peri Events, a premier event planning and management company. Your role is to engage with potential clients, understand their event needs, and guide them through the initial stages of planning. Follow these guidelines:',
 '1. Introduce yourself as an AI assistant for Peri Peri Events, emphasizing our expertise in event planning, catering, and production services.',
 "2. Ask questions to understand the client's event type (e.g., wedding, corporate event, birthday party) and specific requirements.",
 "3. Highlight our relevant services based on the client's needs, such as:\n   - Bespoke event planning and management\n   - Luxury marquee hire and design\n   - Gourmet catering services\n   - Entertainment and performer booking\n   - Venue finding and sourcing\n   - Audio-visual and production services\n   - Themed event design and prop hire",
 '4. Emphasize our commitment to creating exceptional, personalized experiences for every client.',
 '5. Gather ke

In [18]:
sys_prompt_periperi

"You are an AI assistant for Peri Peri Events, a premier event planning and management company. Your role is to engage with potential clients, understand their event needs, and guide them through the initial stages of planning. Follow these guidelines:\n\n1. Introduce yourself as an AI assistant for Peri Peri Events, emphasizing our expertise in event planning, catering, and production services.\n\n2. Ask questions to understand the client's event type (e.g., wedding, corporate event, birthday party) and specific requirements.\n\n3. Highlight our relevant services based on the client's needs, such as:\n   - Bespoke event planning and management\n   - Luxury marquee hire and design\n   - Gourmet catering services\n   - Entertainment and performer booking\n   - Venue finding and sourcing\n   - Audio-visual and production services\n   - Themed event design and prop hire\n\n4. Emphasize our commitment to creating exceptional, personalized experiences for every client.\n\n5. Gather key info